## 1 - Data Collection

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/test_reviews.csv')
df

,review,voted_up
0,While it does feel like they needed a bit more...,True
1,Game is asbolutely good. The Night City is som...,True
2,This game has a JoJo reference.,True
3,"Cheers everyone, after 8 years we finally made...",True
4,made my penis to perfection in a call with fri...,True
...,...,...
1995,The game doesn't bring anything new to the tab...,False
1996,pp go smol ( ͡° ͜ʖ ͡°)\n\npp go big (˵ ͡☉ ͜ʖ ͡...,True
1997,"Great characters, nice city, thrilling storyli...",True
1998,So here is my review after all of this time.\n...,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   review    2000 non-null   object
 1   voted_up  2000 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 17.7+ KB


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df_train, df_test = train_test_split(df, random_state=212)
X_train, y_train = df_train['review'], df_train['voted_up']
X_test, y_test = df_test['review'], df_test['voted_up']
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1500,), (1500,), (500,), (500,))

## 2 - EDA

TODO:
- frequency distribution
- total vocab
- word clouds

comparative EDA - compare values in both classes

watch topic 39 video at around 52:00

In [ ]:
"""
total_vocabulary = set(word for review in X_train_tokenized for word in review)
len(total_vocabulary)
print('There are {} unique tokens in the dataset.'.format(len(total_vocabulary)))
"""

## 3 - Pre-Processing

### Tokenization

In [6]:
import nltk
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re
from string import punctuation
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
punctuation_list = list(punctuation) + ['`', '’', '…']

In [8]:
def tokenize(review):
    review = re.sub(r'\[.*?\]', '', review) # remove markdown tags, only needed for Steam reviews
    review = review.translate(str.maketrans('', '', ''.join(punctuation_list))) # remove all punctuation
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+') # tokenize words with only numbers and latin characters
    return tokenizer.tokenize(review.lower())

In [9]:
X_train_tokenized = list(map(tokenize, X_train))
X_test_tokenized = list(map(tokenize, X_test))
len(X_train_tokenized), len(X_test_tokenized)

(1500, 500)

### Stop-Words Removal

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stopwords_list = stopwords.words('english') + punctuation_list
len(stopwords_list)

214

In [12]:
X_train_stopworded = [[word for word in review if word not in stopwords_list] for review in X_train_tokenized]
X_test_stopworded = [[word for word in review if word not in stopwords_list] for review in X_test_tokenized]
len(X_train_stopworded), len(X_test_stopworded)

(1500, 500)

### Lemmatization

In [13]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
lemmatizer = WordNetLemmatizer() 
X_train_lemmatized = [list(map(lemmatizer.lemmatize, review)) for review in X_train_stopworded]
X_test_lemmatized = [list(map(lemmatizer.lemmatize, review)) for review in X_test_stopworded]
len(X_train_lemmatized), len(X_test_lemmatized)

(1500, 500)

### n-grams

In [ ]:
# todo

### PCA

In [ ]:
# todo

### Finalizing

In [15]:
X_train_preprocessed = [' '.join(review) for review in X_train_lemmatized]
X_test_preprocessed = [' '.join(review) for review in X_test_lemmatized]
X_train_split = [review.split(' ') for review in X_train_preprocessed]
X_test_split = [review.split(' ') for review in X_test_preprocessed]
len(X_train_preprocessed), len(X_test_preprocessed)

(1500, 500)

## 4 - Feature Engineering

### Bag of Words

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cv = CountVectorizer()
X_train_bow = pd.DataFrame(cv.fit_transform(X_train_preprocessed).todense(), columns=cv.get_feature_names())
X_test_bow = pd.DataFrame(cv.transform(X_test_preprocessed).todense(), columns=cv.get_feature_names())
X_train_bow.shape, X_test_bow.shape

((1500, 12529), (500, 12529))

### TF-IDF

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
tf = TfidfVectorizer()
X_train_tf = pd.DataFrame(tf.fit_transform(X_train_preprocessed).todense(), columns=tf.get_feature_names())
X_test_tf = pd.DataFrame(tf.transform(X_test_preprocessed).todense(), columns=tf.get_feature_names())
X_train_tf.shape, X_test_tf.shape

((1500, 12529), (500, 12529))

### Document Embeddings

In [94]:
from gensim.sklearn_api import D2VTransformer
from sklearn.preprocessing import MinMaxScaler

In [112]:
vectorizer = D2VTransformer()
scaler = MinMaxScaler((1, 2)) # scaled to prevent negative values, which do not work with Naive Bayes models
X_train_embed = scaler.fit_transform(pd.DataFrame(vectorizer.fit_transform(X_train_split)))
X_test_embed = scaler.transform(pd.DataFrame(vectorizer.transform(X_test_split)))
X_train_embed.shape, X_test_embed.shape

((1500, 100), (500, 100))

## 5 - Model Building and Evaluation

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [50]:
def get_metrics(y_train, y_hat_train, y_test, y_hat_test):
    train_accuracy = accuracy_score(y_train, y_hat_train)
    test_accuracy = accuracy_score(y_test, y_hat_test)
    train_precision = precision_score(y_train, y_hat_train)
    test_precision = precision_score(y_test, y_hat_test)
    train_recall = recall_score(y_train, y_hat_train)
    test_recall = recall_score(y_test, y_hat_test)
    
    print('\t\tAccuracy\tPrecision\tRecall')
    print(f'Training:\t{round(train_accuracy, 2)}\t\t{round(train_precision, 2)}\t\t{round(train_recall, 2)}')
    print(f'Testing:\t{round(test_accuracy, 2)}\t\t{round(test_precision, 2)}\t\t{round(test_recall, 2)}')
    
    return {'train_accuracy':train_accuracy, 'train_precision':train_precision, 'train_recall':train_recall,
            'test_accuracy':test_accuracy, 'test_precision':test_precision, 'test_recall':test_recall}

In [124]:
model_metrics = []

### Basic Model

In [65]:
y_train.value_counts(normalize=True)

True     0.596667
False    0.403333
Name: voted_up, dtype: float64

In [66]:
train_preds = [True]*len(y_train)
test_preds = [True]*len(y_test)

In [125]:
model_name = 'Predict only "Suggested"'
data_name = 'None'
print(f'{model_name}\t{data_name}')
metrics = {'model':model_name, 'data':data_name}
metrics.update(get_metrics(y_train, train_preds, y_test, test_preds))
model_metrics.append(metrics)

Predict only "Suggested"	None
		Accuracy	Precision	Recall
Training:	0.63		0.63		0.93
Testing:	0.63		0.64		0.91


### Baseline Models

In [126]:
models = [('Logistic Regression', LogisticRegression),
          ('Multinomial Naive Bayes', MultinomialNB),
          ('Random Forest', RandomForestClassifier),
          ('Support Vector Machines', SVC)]
datasets = [('Bag of Words', X_train_bow, X_test_bow),
             ('TF-IDF', X_train_tf, X_test_tf),
             ('Document Embeddings', X_train_embed, X_test_embed)]

In [127]:
for model_name, model in models:
    for data_name, X_train, X_test in datasets:
        classifier = model()
        classifier.fit(X_train, y_train)
        train_preds = classifier.predict(X_train)
        test_preds = classifier.predict(X_test)

        print(f'{model_name}\t\t{data_name}')
        metrics = {'model':model_name, 'data':data_name}
        metrics.update(get_metrics(y_train, train_preds, y_test, test_preds))
        print()
        model_metrics.append(metrics)

Logistic Regression		Bag of Words
		Accuracy	Precision	Recall
Training:	0.98		0.97		1.0
Testing:	0.82		0.84		0.87

Logistic Regression		TF-IDF
		Accuracy	Precision	Recall
Training:	0.92		0.89		0.98
Testing:	0.83		0.81		0.95

Logistic Regression		Document Embeddings
		Accuracy	Precision	Recall
Training:	0.63		0.63		0.93
Testing:	0.64		0.65		0.92

Multinomial Naive Bayes		Bag of Words
		Accuracy	Precision	Recall
Training:	0.94		0.93		0.97
Testing:	0.8		0.83		0.86

Multinomial Naive Bayes		TF-IDF
		Accuracy	Precision	Recall
Training:	0.85		0.8		1.0
Testing:	0.73		0.71		0.98

Multinomial Naive Bayes		Document Embeddings
		Accuracy	Precision	Recall
Training:	0.6		0.61		0.92
Testing:	0.61		0.63		0.91

Random Forest		Bag of Words
		Accuracy	Precision	Recall
Training:	1.0		1.0		1.0
Testing:	0.77		0.81		0.82

Random Forest		TF-IDF
		Accuracy	Precision	Recall
Training:	1.0		1.0		1.0
Testing:	0.77		0.78		0.88

Random Forest		Document Embeddings
		Accuracy	Precision	Recall
Training:	1.0		1.0		1.0


In [128]:
model_metrics_df = pd.DataFrame(model_metrics)
model_metrics_df.sort_values(by='test_accuracy', ascending=False)

,model,data,train_accuracy,train_precision,train_recall,test_accuracy,test_precision,test_recall
11,Support Vector Machines,TF-IDF,0.995333,0.992239,1.000000,0.830,0.819718,0.932692
2,Logistic Regression,TF-IDF,0.920667,0.893509,0.984358,0.828,0.810440,0.945513
1,Logistic Regression,Bag of Words,0.983333,0.972826,1.000000,0.816,0.839506,0.871795
4,Multinomial Naive Bayes,Bag of Words,0.938000,0.929336,0.969832,0.802,0.827692,0.862179
8,Random Forest,TF-IDF,0.998000,0.996659,1.000000,0.772,0.779661,0.884615
7,Random Forest,Bag of Words,0.998000,0.997768,0.998883,0.766,0.805643,0.823718
10,Support Vector Machines,Bag of Words,0.824667,0.776224,0.992179,0.752,0.724880,0.971154
5,Multinomial Naive Bayes,TF-IDF,0.847333,0.796263,1.000000,0.734,0.707657,0.977564
9,Random Forest,Document Embeddings,1.000000,1.000000,1.000000,0.672,0.702186,0.823718
3,Logistic Regression,Document Embeddings,0.632000,0.629239,0.932961,0.644,0.651584,0.923077


In [129]:
model_metrics_df.groupby(by='model').mean().sort_values(by='test_accuracy', ascending=False)

,train_accuracy,train_precision,train_recall,test_accuracy,test_precision,test_recall
model,,,,,,
Logistic Regression,0.845333,0.831858,0.972439,0.762667,0.767176,0.913462
Support Vector Machines,0.815333,0.798072,0.973557,0.737333,0.729749,0.936966
Random Forest,0.998667,0.998142,0.999628,0.736667,0.762496,0.844017
Multinomial Naive Bayes,0.795778,0.778821,0.962384,0.716667,0.722820,0.915598
"Predict only ""Suggested""",0.626000,0.625753,0.928492,0.630000,0.644647,0.907051


In [130]:
model_metrics_df.groupby(by='data').mean().sort_values(by='test_accuracy', ascending=False)

,train_accuracy,train_precision,train_recall,test_accuracy,test_precision,test_recall
data,,,,,,
TF-IDF,0.940333,0.919668,0.996089,0.791,0.779369,0.935096
Bag of Words,0.936000,0.919038,0.990223,0.784,0.799430,0.882212
Document Embeddings,0.715000,0.716464,0.944693,0.640,0.657882,0.890224
None,0.626000,0.625753,0.928492,0.630,0.644647,0.907051


### Pipelines

Don't need to do this for preliminaries, just add this into final notebooks.

In [ ]:
"""
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
"""

In [ ]:
"""
rf =  Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Random Forest', RandomForestClassifier(n_estimators=100))])
svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])
"""

In [ ]:
"""
models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]
"""

In [ ]:
"""
scores = [(name, cross_val_score(model, X_train_tokenized, y_train.values, cv=2).mean()) for name, model, in models]
scores
"""

### Gridsearch

Don't need to do this for preliminaries, just add this into final notebooks.